In [ ]:
!pip install tensorflow
!pip install catboost

In [35]:
import pandas as pd
import numpy as np
import nltk
import re, string
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

from tensorflow.keras.layers import Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,GRU
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
from google.colab import data_table
data_table.enable_dataframe_formatter()
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# setting dir
import os
os.chdir('/content/drive/MyDrive/Projects/Hacks/WWC Hackathon for Social Good/wwcode-hackathon-2022/data')

In [7]:
%ls -l

total 681107
-rw------- 1 root root     17507 Jul  3 00:14 bigram_sentiment_freq.csv
-rw------- 1 root root  23767008 Jun 29 22:51 climate_data_combined_files.xlsx
-rw------- 1 root root  45953823 Jun 30 15:17 climate_data_jean.csv
-rw------- 1 root root   3152039 Jun 27 20:54 climate-fever-dataset-r1.jsonl
drwx------ 2 root root      4096 Jun 28 17:09 climate-new-db-dataset/
-rw------- 1 root root 100557567 Jun 28 17:08 climate-new-db-dataset.zip
-rw------- 1 root root  56280594 Jun 29 21:58 climate-news-db-all.csv
-rw------- 1 root root         0 Jun 30 14:22 db.sqlite
drwx------ 2 root root      4096 Jun 28 17:29 fake-news/
-rw------- 1 root root  48725919 Jun 28 17:28 fake-news.zip
-rw------- 1 root root 135969792 Jul  2 04:10 news_cleaned_tokens.csv
-rw------- 1 root root  33995123 Jul  3 01:20 submission_with_ML_labels.csv
-rw------- 1 root root  25046463 Jul  2 17:53 test_jean.csv
-rw------- 1 root root    152161 Jun 27 20:48 usa_dataset.xlsx
-rw------- 1 root root 136239250 Jul

In [22]:
# Reading data from csv
fn_train = pd.read_csv("fake-news/train.csv") # fake news train
fn_test = pd.read_csv("fake-news/test.csv") # fake news test

climate_df  = pd.read_csv("news_cleaned_tokens.csv") # climate data to predict on
print(fn_train.shape)
print(fn_test.shape)
print(climate_df.shape)

(20800, 5)
(5200, 4)
(9352, 10)


In [23]:
# preprocessing the train dataset
print(fn_train.isna().sum())
print(fn_test.isna().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
id          0
title     122
author    503
text        7
dtype: int64


In [24]:
train = fn_train[['text','label']].fillna('')
test = fn_test[['text']].fillna('')

print(train.isna().sum())
print(test.isna().sum())

text     0
label    0
dtype: int64
text    0
dtype: int64


In [69]:
# Performing data preprocessing on column 'text' of fake news dataset 
# This step was already performed on climate data
def remove_noise(tweet_tokens, stop_words = ()):
  """
  This function removes noise such as hyperlinks, special symbols, and punctuation.
  It also applies lemmatization given the tag of each token.
  
  """
  cleaned_tokens = []
  for token, tag in pos_tag(tweet_tokens):
      # convert text to lowercase
      token = token.lower()
      # remove hyperlinks
      token = re.sub('https?:\/\/[^\s\n\r]+','',token)
      # remove symbols and punctuations
      #token = re.sub("(?:[\$\-_@\.&+#%^=?`~]|[*\(\),\[\]{}\\\/|;:\'\"<>]|)",'',token)
      # or remove all non alpha-numeric? 
      token = re.sub("[^a-zA-Z0-9!?]","", token)
      if tag.startswith("NN"):
          pos = 'n'
      elif tag.startswith('VB'):
          pos = 'v'
      else:
          pos = 'a'

      lemmatizer = WordNetLemmatizer()
      token = lemmatizer.lemmatize(token, pos)

      if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
          cleaned_tokens.append(token.lower())
  return cleaned_tokens

In [59]:
# token words for preprocessing
def tokenize_text(text_list):
  tokenized_text = []
  for text in text_list:
    tokenized_words = word_tokenize(text)  
    tokenized_text.append(tokenized_words)
  return tokenized_text

In [71]:
#add all the fake news train/test data into a list
train_list = [news for news in train['text']]
test_list = [news for news in test['text']]

# tokenize words
train_token_list = tokenize_text(train_list)
test_token_list = tokenize_text(test_list)

In [72]:
# set stopwords
stop_words = stopwords.words('english')

In [73]:
train_cleaned_tokens = []
test_cleaned_tokens = []
for news in train_token_list:
  train_cleaned_tokens.append(remove_noise(news,stop_words))

for news in test_token_list:
  test_cleaned_tokens.append(remove_noise(news, stop_words))
%time

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [61]:
# tokenize words
tokenized_text = []
for text in train_list[:1]:
  tokenized_words = word_tokenize(text)  
  tokenized_text.append(tokenized_words)

In [75]:
fn_train.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [78]:
# add cleaned tokens to fake news dataset
fn_train['cleaned_tokens'] = train_cleaned_tokens
fn_test['cleaned_tokens'] = test_cleaned_tokens

# add cleaned tokens as a string to df
def token_to_str(token_list):
  return ' '.join(str(t) for t in token_list)

fn_train['cleaned_text'] = fn_train['cleaned_tokens'].apply(token_to_str)
fn_test['cleaned_text'] = fn_test['cleaned_tokens'].apply(token_to_str)
print(fn_train.columns)
print(fn_train.shape)
print(fn_test.shape)

Index(['id', 'title', 'author', 'text', 'label', 'cleaned_tokens',
       'cleaned_text'],
      dtype='object')
(20800, 7)
(5200, 6)


In [85]:
# save cleaned data
# fn_train.to_csv('fn_train_cleaned.csv', index=False)
# fn_test.to_csv('fn_test_cleaned.csv', index=False)